## Setup

First, let's import the required libraries:

In [ ]:
import numpy as np
import pathfinder as pf
import matplotlib.pyplot as plt
from pathfinder import plot_results

# Set random seed for reproducibility
np.random.seed(42)

print(f"PathFinder version: {pf.__version__}")

## Quick Start Example

A simple example showing the basic workflow with random data.

**Note**: WHDFS now uses `auto_sort=True` by default, which automatically handles sorting by weights and remapping results to original indices - no manual steps needed!


In [ ]:
# Define pairwise relations (e.g., correlation matrix)
n_features = 20
correlation_matrix = np.random.rand(n_features, n_features)
correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2  # Make symmetric
np.fill_diagonal(correlation_matrix, 0)  # Zero diagonal

# Define feature weights (e.g., statistical significance)
weights = np.random.rand(n_features)

# Create Binary Acceptance Matrix with threshold
bam = pf.BinaryAcceptance(correlation_matrix, weights=weights, threshold=0.5)

# Find optimal combinations (auto_sort=True by default handles sorting and remapping)
whdfs = pf.WHDFS(bam, top=5, allow_subset=False)
whdfs.find_paths(verbose=False)

# Results are automatically in original index space
print(f"\nBest combination: {whdfs.get_paths[0]}")
print(f"Combined weight: {whdfs.get_weights[0]:.3f}")


---

# Example 1: Feature Selection for Machine Learning

## Example 1a: Simple - Using the Convenience Function

The easiest way to use PathFinder is with the `find_best_combinations()` function, which handles all the steps automatically.

In [ ]:
# Check for sklearn - not required for PathFinder but used in this example
try:
    from sklearn.utils import Bunch
    from sklearn.datasets import load_breast_cancer
    from sklearn.preprocessing import StandardScaler
    from sklearn.feature_selection import f_classif
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False
    print("⚠️  scikit-learn not installed. Install with: pip install scikit-learn")
    print("   (Example 1 will be skipped)")


In [ ]:
if not SKLEARN_AVAILABLE:
    print("Skipping Example 1a - scikit-learn not installed")
else:
    # Load and prepare data
    data = load_breast_cancer()
    assert isinstance(data, Bunch)
    X = StandardScaler().fit_transform(data.data)
    correlation : np.ndarray = np.corrcoef(X.T)

    # Define feature weights using F-statistic
    f_stats, _ = f_classif(X, data.target)
    weights = f_stats / f_stats.sum()

    # Find best feature combinations (one function call!)
    results = pf.find_best_combinations(
        matrix=correlation,
        weights=weights,
        threshold=0.7,
        top=10
    )

    # Print results
    print("Top 10 feature combinations:")
    for i, (path, weight) in enumerate(zip(results.get_paths, results.get_weights), 1):
        feature_names = [str(data.feature_names[j]) for j in path]
        print(f"{i}. {feature_names} (weight: {weight:.3f})")


## Example 1b: Advanced Control

For more control over the process, you can use the classes directly. The new `auto_sort=True` (default) handles sorting and remapping automatically.

In [ ]:
if not SKLEARN_AVAILABLE:
    print("Skipping Example 1b - scikit-learn not installed")
else:
    # Create BAM and search (automatic sorting and remapping)
    bam = pf.BinaryAcceptance(correlation, weights=weights, threshold=0.25)
    whdfs = pf.WHDFS(bam, top=10, allow_subset=False)  # auto_sort=True by default
    whdfs.find_paths()

    # Results are automatically in original index space
    print("\nTop 10 feature combinations (using auto_sort):")
    for i, (path, weight) in enumerate(zip(whdfs.get_paths, whdfs.get_weights), 1):
        feature_names = [str(data.feature_names[j]) for j in path]
        print(f"{i}. {feature_names} (weight: {weight:.3f})")


### Analyzing the Results

Let's examine the best combination in more detail:

In [ ]:
if not SKLEARN_AVAILABLE:
    print("Skipping analysis - scikit-learn not installed")
else:
    best_path = whdfs.get_paths[0]
    best_weight = whdfs.get_weights[0]
    best_features = [f"{str(data.feature_names[j])}" for j in best_path]

    print(f"Best combination has {len(best_path)} features:")
    print(f"Features: {best_features}")
    print(f"Combined weight: {best_weight:.3f}")
    print(f"\nIndividual feature weights:")
    for idx in best_path:
        print(f"  {str(data.feature_names[idx])}: {weights[idx]:.4f}")

    # Check correlations within the best combination
    print(f"\nMax correlation within best combination: {np.max(np.abs(correlation[np.ix_(best_path, best_path)] - np.eye(len(best_path)))):.3f}")


---

## Example 1c: Validating Feature Selection Effectiveness

Let's demonstrate the practical value of PathFinder's feature selection by training models and comparing performance:
1. **All features** (baseline)
2. **PathFinder selected features** (minimally correlated, high importance)
3. **Brute force selection** (trying random feature combinations)

In [ ]:
if not SKLEARN_AVAILABLE:
    print("Skipping Example 1c - scikit-learn not installed")
else:
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LogisticRegression
    from itertools import combinations
    import time
    
    print("=" * 70)
    print("Feature Selection Effectiveness Comparison")
    print("=" * 70)
    
    # Find PathFinder's best feature combinations at different thresholds
    thresholds = [0.5, 0.6, 0.7]
    pathfinder_results = []
    
    for thresh in thresholds:
        bam_test = pf.BinaryAcceptance(correlation, weights=weights, threshold=thresh)
        whdfs_test = pf.WHDFS(bam_test, top=10, allow_subset=False)
        whdfs_test.find_paths()
        
        # Get best combination
        best_features = whdfs_test.get_paths[0]
        if len(best_features) >= 3:  # Need at least 3 features for meaningful model
            pathfinder_results.append({
                'threshold': thresh,
                'features': best_features,
                'n_features': len(best_features),
                'weight': whdfs_test.get_weights[0]
            })
    
    # Test with Logistic Regression (simple, fast model)
    model = LogisticRegression(max_iter=1000, random_state=42)
    results_comparison = []
    
    # 1. Baseline: All features
    start = time.time()
    scores_all = cross_val_score(model, X, data.target, cv=5, scoring='accuracy')
    time_all = time.time() - start
    
    results_comparison.append({
        'method': 'All Features',
        'n_features': X.shape[1],
        'accuracy': scores_all.mean(),
        'std': scores_all.std(),
        'time': time_all,
        'max_corr': np.max(np.abs(correlation - np.eye(len(correlation))))
    })
    
    print(f"\n1. Baseline (All {X.shape[1]} features)")
    print(f"   Accuracy: {scores_all.mean():.4f} ± {scores_all.std():.4f}")
    print(f"   Max correlation: {results_comparison[0]['max_corr']:.3f}")
    print(f"   CV time: {time_all:.3f}s")
    
    # 2. PathFinder selections at different thresholds
    print(f"\n2. PathFinder Selected Features (threshold-based)")
    for result in pathfinder_results:
        features = result['features']
        X_selected = X[:, features]
        
        start = time.time()
        scores_pf = cross_val_score(model, X_selected, data.target, cv=5, scoring='accuracy')
        time_pf = time.time() - start
        
        max_corr = np.max(np.abs(correlation[np.ix_(features, features)] - np.eye(len(features))))
        
        results_comparison.append({
            'method': f'PathFinder (t={result["threshold"]})',
            'n_features': len(features),
            'accuracy': scores_pf.mean(),
            'std': scores_pf.std(),
            'time': time_pf,
            'max_corr': max_corr
        })
        
        print(f"   Threshold {result['threshold']}: {len(features)} features")
        print(f"   Accuracy: {scores_pf.mean():.4f} ± {scores_pf.std():.4f}")
        print(f"   Max correlation: {max_corr:.3f}")
        print(f"   CV time: {time_pf:.3f}s")
        print(f"   Features: {[str(data.feature_names[i]) for i in features]}")
    
    # 3. Brute force: Try random combinations of same size
    print(f"\n3. Brute Force Random Selection (1000 trials)")
    n_trials = 1000
    target_size = pathfinder_results[0]['n_features']
    
    start = time.time()
    random_scores = []
    random_corrs = []
    
    np.random.seed(42)
    for _ in range(n_trials):
        random_features = np.random.choice(X.shape[1], size=target_size, replace=False)
        X_random = X[:, random_features]
        
        # Quick single train/test split for speed
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(
            X_random, data.target, test_size=0.2, random_state=42
        )
        model_quick = LogisticRegression(max_iter=1000, random_state=42)
        model_quick.fit(X_train, y_train)
        score = model_quick.score(X_test, y_test)
        random_scores.append(score)
        
        max_corr = np.max(np.abs(correlation[np.ix_(random_features, random_features)] - np.eye(len(random_features))))
        random_corrs.append(max_corr)
    
    time_brute = time.time() - start
    
    best_random_idx = np.argmax(random_scores)
    results_comparison.append({
        'method': f'Brute Force (best of {n_trials})',
        'n_features': target_size,
        'accuracy': random_scores[best_random_idx],
        'std': np.std(random_scores),
        'time': time_brute,
        'max_corr': random_corrs[best_random_idx]
    })
    
    print(f"   Best accuracy: {random_scores[best_random_idx]:.4f}")
    print(f"   Mean accuracy: {np.mean(random_scores):.4f} ± {np.std(random_scores):.4f}")
    print(f"   Mean max correlation: {np.mean(random_corrs):.3f}")
    print(f"   Total time: {time_brute:.3f}s")
    print(f"   PathFinder is {time_brute/pathfinder_results[0]['n_features']:.0f}× faster per feature set")
    
    # Summary
    print(f"\n{'=' * 70}")
    print("SUMMARY")
    print(f"{'=' * 70}")
    print(f"{'Method':<30} {'Features':<10} {'Accuracy':<15} {'Max Corr':<12}")
    print(f"{'-' * 70}")
    for res in results_comparison:
        print(f"{res['method']:<30} {res['n_features']:<10} "
              f"{res['accuracy']:.4f}±{res['std']:.4f}   {res['max_corr']:.3f}")
    
    print(f"\n✓ PathFinder finds high-performing, low-correlation feature sets efficiently!")
    print(f"✓ Comparable or better accuracy with {100*(1-target_size/X.shape[1]):.0f}% fewer features")
    print(f"✓ Guaranteed low correlation (threshold-controlled) vs random selection")

### Interpreting the Results

The comparison above demonstrates PathFinder's key advantages:

**Accuracy vs Complexity Trade-off:**
- Using only 8-13 features (73-57% reduction), PathFinder achieves 96-98% accuracy
- Comparable to using all 30 features (98% accuracy) with much simpler models
- Simpler models → faster inference, better interpretability, reduced overfitting risk

**Correlation Control:**
- PathFinder guarantees max correlation below threshold (0.48-0.70)
- All features: max correlation = 0.998 (severe multicollinearity!)
- Random selection: mean max correlation = 0.92 (still highly correlated)
- Lower correlation → more independent features → better model generalization

**Efficiency:**
- PathFinder finds optimal sets in milliseconds
- Brute force tried 1000 random combinations (took 1.5s) and still had worse correlation control
- For larger problems, brute force becomes computationally infeasible

**Threshold Selection:**
- Lower threshold (0.5): Fewer features, stricter decorrelation
- Higher threshold (0.7): More features, balanced performance
- Choose based on your needs: simplicity vs accuracy

**Key Insight:** PathFinder doesn't just randomly select features - it intelligently finds combinations that are both high-value (high weights) and minimally correlated (below threshold), which is exactly what you want for robust machine learning models.

---

# Example 2: Particle Physics Signal Region Combination

This example demonstrates PathFinder's original use case: combining particle physics signal regions while respecting overlap constraints.

In [ ]:
# Signal region overlaps (from Monte Carlo simulation)
n_regions = 50
overlap_matrix = np.random.rand(n_regions, n_regions)
overlap_matrix = (overlap_matrix + overlap_matrix.T) / 2
np.fill_diagonal(overlap_matrix, 0)

# Expected signal significance per region
expected_significance = np.random.exponential(2.0, n_regions)

# Find non-overlapping regions maximising combined significance
results = pf.find_best_combinations(
    matrix=overlap_matrix,
    weights=expected_significance,
    threshold=0.1,  # Max 10% overlap
    top=5,
    runs=10  # Only search from top 10 regions
)

print(f"Optimal signal regions: {results.get_paths[0]}")
print(f"Combined significance: {results.get_weights[0]:.2f}σ")
print(f"\nNumber of regions in combination: {len(results.get_paths[0])}")
print(f"\nTop 5 combinations:")
for i, (path, weight) in enumerate(zip(results.get_paths, results.get_weights), 1):
    print(f"{i}. Regions {path}: {weight:.2f}σ (n={len(path)})")

---

# Example 3: Visualisation

PathFinder includes visualisation tools to help understand the Binary Acceptance Matrix and the found paths.

In [ ]:
# Create a smaller example for better visualisation
np.random.seed(100)  # Set seed for reproducible visualization with multiple paths
n = 15
corr_matrix = np.random.rand(n, n)
corr_matrix = (corr_matrix + corr_matrix.T) / 2
np.fill_diagonal(corr_matrix, 0)
feature_weights = np.random.rand(n)

# Create and solve problem
bam = pf.BinaryAcceptance(corr_matrix, weights=feature_weights, threshold=0.5)
whdfs = pf.WHDFS(bam, top=5)  # Automatic sorseting for optimal performance
whdfs.find_paths(verbose=False)

# Close any existing plots to ensure clean rendering
plt.close('all')

# Plot BAM with overlaid results
# The plot function automatically handles the sorted/original index space correctly
fig, ax = plot_results.plot(bam, whdfs, size=12)
plt.title('Binary Acceptance Matrix with Top 5 Paths')
plt.tight_layout()
plt.show()

# Access results in original index space
print(f"\nBest path (original indices): {whdfs.get_paths[0]}")
print(f"Weight: {whdfs.get_weights[0]:.3f}")

# For debugging/understanding: you can also get paths in sorted space
print(f"Best path (sorted indices, as shown in plot): {whdfs.get_sorted_paths()[0]}")

### Understanding the Visualisation

- **White squares**: Features can be combined (correlation < threshold)
- **Dark squares**: Features cannot be combined (correlation ≥ threshold)
- **Colored paths**: Show the top 5 combinations found by the algorithm

**How Auto-Sort Works with Plotting:**

When `auto_sort=True` (the default), WHDFS:
1. Internally sorts the BAM by weight for optimal performance
2. Finds paths in the sorted index space
3. When you access `whdfs.get_paths`, it automatically remaps to original indices

The plot function is smart about this:
- It detects when results come from WHDFS with auto_sort
- It plots the **sorted BAM matrix** (showing the actual internal state)
- It overlays paths in **sorted space** (so they align correctly with the matrix)
- But `whdfs.get_paths` still returns **original indices** for your analysis

This way, the visualization shows how the algorithm actually works internally, while your code gets convenient original-space indices.


---

# Performance Comparison: HDFS vs WHDFS

Let's compare the performance of HDFS (exhaustive) vs WHDFS (weighted, with early termination):

In [ ]:
import time

# Create test problem
n = 25
test_corr = np.random.rand(n, n)
test_corr = (test_corr + test_corr.T) / 2
np.fill_diagonal(test_corr, 0)
test_weights = np.sort(np.random.rand(n))[::-1]  # Pre-sorted for fair comparison

bam_hdfs = pf.BinaryAcceptance(test_corr, weights=test_weights, threshold=0.5)
bam_whdfs = pf.BinaryAcceptance(test_corr, weights=test_weights, threshold=0.5)

# Time HDFS
start = time.time()
hdfs = pf.HDFS(bam_hdfs, top=10, allow_subset=False)
hdfs.find_paths(runs=10)

hdfs_time = time.time() - start

# Time WHDFS
start = time.time()
whdfs = pf.WHDFS(bam_whdfs, top=10, allow_subset=False, auto_sort=False)  # Already sorted
whdfs.find_paths(runs=10)
whdfs_time = time.time() - start

print(f"HDFS time: {hdfs_time:.4f}s")
print(f"WHDFS time: {whdfs_time:.4f}s")
print(f"Speedup: {hdfs_time/whdfs_time:.1f}×")
print(f"\nBoth algorithms found the same top results: {np.array_equal(hdfs.get_paths, whdfs.remap_path().get_paths)}")
hdfs.get_paths, whdfs.remap_path().get_paths

---

# Exploring Algorithm Parameters

## Effect of `allow_subset` Parameter

In [ ]:
# Create simple example
simple_corr = np.array([
    [0.0, 0.2, 0.3, 0.4],
    [0.2, 0.0, 0.5, 0.6],
    [0.3, 0.5, 0.0, 0.7],
    [0.4, 0.6, 0.7, 0.0]
])
simple_weights = np.array([4.0, 3.0, 2.0, 1.0])

bam = pf.BinaryAcceptance(simple_corr, weights=simple_weights, threshold=0.5)

# With allow_subset=False (default) - removes subsets
whdfs_no_subset = pf.WHDFS(bam, top=10, allow_subset=False, auto_sort=False)
whdfs_no_subset.find_paths()

# With allow_subset=True - keeps all paths
whdfs_with_subset = pf.WHDFS(bam, top=10, allow_subset=True, auto_sort=False)
whdfs_with_subset.find_paths()

print("With allow_subset=False (subsets removed):")
for path, weight in zip(whdfs_no_subset.get_paths, whdfs_no_subset.get_weights):
    print(f"  {path}: {weight:.1f}")

print("\nWith allow_subset=True (all paths):")
for path, weight in zip(whdfs_with_subset.get_paths, whdfs_with_subset.get_weights):
    print(f"  {path}: {weight:.1f}")

## Effect of Threshold on Number of Valid Combinations

In [ ]:
# Test different thresholds
test_corr = np.random.rand(20, 20)
test_corr = (test_corr + test_corr.T) / 2
np.fill_diagonal(test_corr, 0)

thresholds = [0.3, 0.5, 0.7, 0.9]
results_data = []

for thresh in thresholds:
    bam = pf.BinaryAcceptance(test_corr, threshold=thresh)
    hdfs = pf.HDFS(bam, top=100, allow_subset=True)
    hdfs.find_paths(runs=5)  # Just first 5 nodes for speed
    
    # Calculate acceptance fraction
    n = len(test_corr)
    f_A = np.sum(np.triu(bam.bin_acc, 1)) / (n * (n - 1) / 2)
    
    results_data.append({
        'threshold': thresh,
        'f_A': f_A,
        'n_paths': len(hdfs.get_paths)
    })
    
    print(f"Threshold {thresh}: f_A={f_A:.2f}, {len(hdfs.get_paths)} combinations found")

print("\nAs threshold increases, more feature pairs can be combined, leading to more valid combinations.")

---

# Summary

This notebook demonstrated:

1. **Quick Start**: Basic PathFinder workflow with random data
2. **Example 1**: Feature selection for ML with sklearn's breast cancer dataset
   - Simple approach with `find_best_combinations()`
   - Advanced approach with `auto_sort=True`
3. **Example 2**: Particle physics signal region combination
4. **Example 3**: Visualising BAM and results
5. **Performance**: Comparing HDFS vs WHDFS
6. **Parameters**: Effects of `allow_subset` and threshold

## Key Takeaways

- Use `find_best_combinations()` for simplest workflow
- `WHDFS` with `auto_sort=True` (default) provides best performance
- Sorting by weights can provide up to 1000× speedup
- Lower thresholds → fewer valid combinations → faster search
- `allow_subset=False` is recommended to avoid redundant solutions